In [1]:
import numpy as np 
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import copy 
import pandas as pd
import time
import datetime

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
np.random.seed(7)

Using TensorFlow backend.


# Data Preprocessing

In [9]:
Dataset = r'../scripts/Data/TexasBlackout.csv'
fulldata = pd.read_csv(Dataset) 

In [10]:
fulldata

,ID,County,Customers Tracked,Customers Out,Outage %,DateTime
0,1510,Dallam,"5,727",0,0%,2021-02-17 01:55:55.281883
1,1734,Sherman,"2,992",0,0%,2021-02-17 01:55:58.352994
2,1762,Hansford,"5,098",0,0%,2021-02-17 01:56:01.411124
3,1801,Ochiltree,"8,959",0,0%,2021-02-17 01:56:04.414076
4,1858,Lipscomb,"4,164",0,0%,2021-02-17 01:56:06.654780
...,...,...,...,...,...,...
204138,1581,Camp,"8,126",0,0%,2021-02-26 19:39:04.719373
204139,1454,Rains,"11,095",0,0%,2021-02-26 19:39:06.938395
204140,1329,Aransas,"19,503",0,0%,2021-02-26 19:39:10.250157
204141,1481,Travis,"627,720",6,0%,2021-02-26 19:39:13.562152


In [21]:
fulldata['Customers Out'].describe()

count     204143
unique      9427
top            0
freq      103689
Name: Customers Out, dtype: object

In [11]:
fulldata.keys()

Index(['ID', 'County', 'Customers Tracked', 'Customers Out', 'Outage %',
       'DateTime'],
      dtype='object')

In [3]:
# data=fulldata[['dataid','local_15min','grid']]

In [12]:
sorteddata=fulldata.sort_values(by = ['ID', 'DateTime'])

In [13]:
ids=sorteddata['ID'].unique().tolist()

In [15]:
print(len(ids))

253


In [17]:
blackout_data = []
# def convertDate(d):
#     d = pd.to_datetime(d[:-3])
#     return d 

for i in range(len(ids)):
    blackout_data.append(sorteddata.loc[sorteddata.ID==ids[i]])
    blackout_data[i] = blackout_data[i].reset_index().drop(columns=['index'])
    # blackout_data[i]['DateTime'] = blackout_data[i]['DateTime'].apply(convertDate)
    #Convert datetimes to ints for faster ploting 
    # blackout_data[i]['15min_ints'] =  blackout_data[i]['local_15min'].map(mdates.date2num)

In [18]:
blackout_data[0]

,ID,County,Customers Tracked,Customers Out,Outage %,DateTime
0,1326,Anderson,"24,799",525,2.12%,2021-02-17 02:02:29.849683
1,1326,Anderson,"24,799",525,2.12%,2021-02-17 02:15:30.773378
2,1326,Anderson,"24,799",549,2.21%,2021-02-17 02:28:25.200753
3,1326,Anderson,"24,799",549,2.21%,2021-02-17 02:41:24.546364
4,1326,Anderson,"24,799",549,2.21%,2021-02-17 02:54:50.976373
...,...,...,...,...,...,...
803,1326,Anderson,"24,800",0,0%,2021-02-26 18:19:46.974491
804,1326,Anderson,"24,800",0,0%,2021-02-26 18:38:09.083511
805,1326,Anderson,"24,800",0,0%,2021-02-26 18:56:12.414498
806,1326,Anderson,"24,800",0,0%,2021-02-26 19:14:41.210969


In [19]:
blackout_data[1]

,ID,County,Customers Tracked,Customers Out,Outage %,DateTime
0,1327,Andrews,"10,567","2,712",25.66%,2021-02-17 02:00:39.111638
1,1327,Andrews,"10,567","2,712",25.66%,2021-02-17 02:13:42.159264
2,1327,Andrews,"10,567","1,141",10.8%,2021-02-17 02:26:36.117825
3,1327,Andrews,"10,567","1,141",10.8%,2021-02-17 02:39:29.229583
4,1327,Andrews,"10,567","1,141",10.8%,2021-02-17 02:53:00.754098
...,...,...,...,...,...,...
805,1327,Andrews,"10,567",1,0.01%,2021-02-26 18:17:54.995855
806,1327,Andrews,"10,567",1,0.01%,2021-02-26 18:36:14.476778
807,1327,Andrews,"10,567",1,0.01%,2021-02-26 18:54:19.630771
808,1327,Andrews,"10,567",1,0.01%,2021-02-26 19:12:44.665062
